In [1]:
import os
import os.path
import xarray as xr 
import numpy as np
import glob
import functions as func
import calendar as calendar

In [2]:
pa = "/project/mojave/cmip6/{historical,ssp370}/{Amon,Omon}/psl/CESM2/r1i1p1f1/gn/*.nc"
si = pa.find('{')
if si != -1:    # { } syntax found, search accordingly
    ei = pa.find('}')
    fl = pa[si+1:ei].split(',')
    print(si)
    print(ei)
    files_list = []
    for ext in fl:
        print(pa[:si]+ext+pa[ei+1:])
        test = pa[:si]+ext+pa[ei+1:]
        si2 = test.find('{')
        if si2 != -1:
            print("More than one set of { } syntax found in entry in namelist, only one set of { } allowed per row, exiting")
            exit()
        files_list.extend(glob.glob(pa[:si]+ext+pa[ei+1:]))
else:    
    files_list = glob.glob(pa)

22
40
/project/mojave/cmip6/historical/{Amon,Omon}/psl/CESM2/r1i1p1f1/gn/*.nc
More than one set of { } syntax found in entry in namelist, only one set of { } allowed per row, exiting
/project/mojave/cmip6/ssp370/{Amon,Omon}/psl/CESM2/r1i1p1f1/gn/*.nc
More than one set of { } syntax found in entry in namelist, only one set of { } allowed per row, exiting
[]


In [2]:
# not using netCDF operator ncea here as data for one run (for one variable) can be strung across multiple files

vlist = ['prect','psl','trefht','ts','ssh']

for vn in vlist:
    names = []
    paths = []
    syear = []
    eyear = []
    names_EM = []
    EM_num = []
    
#    print(vn)
    with open('../namelist_byvar/namelist_'+vn) as f:
        for line in f:
            check = line.split(' | ')
            if check[0].rstrip('\r\n')=='missing':
                names.append(check[0].rstrip('\r\n'))
                paths.append(check[0].rstrip('\r\n'))
                syear.append(check[0].rstrip('\r\n'))
                eyear.append(check[0].rstrip('\r\n'))
                EM_num.append(-1)
                names_EM.append(check[0].rstrip('\r\n'))
            else:
                names.append(check[0])
                paths.append(check[1])
                syear.append(check[2])
                eyear.append(check[3])
                ems = check[4].split('-')
                EM_num.append(ems[0])
                names_EM.append(ems[1].rstrip('\r\n'))

    EM_numI = np.array(EM_num).astype(int)
    EM_numI_max = EM_numI.max()
    print(type(EM_numI_max))
    
    for x in range(1, EM_numI_max+1):
        rn = np.array(paths)[np.where(EM_numI == x)]
        rn_syear = np.array(syear)[np.where(EM_numI == x)]
        rn_eyear = np.array(eyear)[np.where(EM_numI == x)]
        rn_names_EM = np.array(names_EM)[np.where(EM_numI == x)] 
        
        fno = rn_names_EM[0].rstrip('\r\n')+'.cvdp_data.'+vn+'.ensemble_mean.'+rn_syear[0]+'-'+rn_eyear[0]+'.nc'
        fno = fno.replace(' ','_')
        print(fno)
        if os.path.exists(fno):
            print(fno+" already created")
            break

        if not all(y==rn_syear[0] for y in rn_syear):
            print("Not all start years are the same for the "+rn_names_EM[0].rstrip('\r\n')+" ensemble for "+vn+", not creating ensemble mean, exiting")
            break
        if not all(y==rn_eyear[0] for y in rn_eyear):
            print("Not all end years are the same for the "+rn_names_EM[0].rstrip('\r\n')+" ensemble for "+vn+", not creating ensemble mean, exiting")
            break

        gg = 0        
        for mr in rn:
            fils2 = glob.glob(mr)
#            print('Reading in: '+rn_syear[gg]+' '+rn_eyear[gg]+' '+vn)
            farr = func.data_read_in_3D(fils2,rn_syear[gg],rn_eyear[gg],vn)
            eeeeeeeeee
            if (gg==0):
                em = xr.zeros_like(farr)
                em_cntr = xr.zeros_like(farr)
                source_attrs = farr.attrs
            em = em+farr
            em_cntr = em_cntr+xr.where(farr.isnull()==False,1,0)   
            print(np.min(em_cntr))
            gg = gg+1

#        print(np.max(em))
#        em = em/float(gg)
        em = em/em_cntr
#        print(np.max(em))
        
        em.attrs = source_attrs
#        print(em)
        em.name = vn
        em.to_netcdf(fno,encoding={vn:{'dtype':'float32'}, 'lat':{'dtype':'float32','_FillValue': None}, 'lon':{'dtype':'float32','_FillValue': None}})

NCAR.cvdp_data.prect.ensemble_mean.1979-2018.nc
('PRECC', 'PRECL', 'PRECT', 'pr', 'PPT', 'ppt', 'p', 'P', 'precip', 'PRECIP', 'tp', 'prcp', 'prate')
<xarray.DataArray 'pr' (time: 3012, lat: 192, lon: 288)>
dask.array<open_dataset-2fe5182f4f9a43ea4e824c922ae7fd72pr, shape=(3012, 192, 288), dtype=float32, chunksize=(3012, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1850-01-16 12:00:00 ... 2100-12-16 12:00:00
Attributes:
    standard_name:   precipitation_flux
    long_name:       precipitation_flux
    units:           kg m-2 s-1
    comment:         Created using NCL code
    cell_methods:    time: mean (interval: 1 month)
    history:         (PRECC+PRECL)*r[h2o]
    original_units:  m-1 s-1
    original_name:   PRECC, PRECL
    _FillValue:      1e+20


NameError: name 'eeeeeeeeee' is not defined

In [4]:
for x in range(1, 5):
    print(x)

1
2
3
4
